Datei einlesen

In [ ]:
import pandas as pd
from datetime import datetime, date, timedelta

data = pd.read_csv("https://raw.githubusercontent.com/kfreyburger/sap/main/dwc/v01/D6CaseScd/GbiCustScdForJupyter.csv", sep = ";", header = 0, parse_dates=['ValidFrom','ValidTo'])
data.head()


,SurrogatId,BusinessKey,Description,Currency,Salesorg,Country,ScdVersion,Current,ValidFrom,ValidTo
0,1,1000,Rocky Mountain Bikes,USD,UW00,US,1,1,1900-01-01,2099-12-31
1,2,6000,Windy City Bikes,USD,UE00,US,1,1,1900-01-01,2099-12-31
2,3,7000,Furniture City Bikes,USD,UE00,US,1,1,1900-01-01,2099-12-31
3,0,6000,Windy City Bikes,USD,UW00,US,0,0,2014-01-01,2099-12-31
4,0,7000,Furniture City Bikes,USD,UE00,US,0,0,1900-01-01,2099-12-31


In [ ]:
data.dtypes

SurrogatId              int64
BusinessKey             int64
Description            object
Currency               object
Salesorg               object
Country                object
ScdVersion              int64
Current                 int64
ValidFrom      datetime64[ns]
ValidTo        datetime64[ns]
dtype: object

In [ ]:
surrogatnext = int(data["SurrogatId"].max())+1
# Daten der Quelle
df_source = data[data["SurrogatId"]==0]

# bereits vorhandene Einträge zur Quelle
df_dim = data[data["Current"]==1]
df_dim = df_dim[df_dim["BusinessKey"].isin(df_source["BusinessKey"])]

# hier werden die geänderten Einträge gesammelt
rows=[]
#  Iteration über die Einträge der Quelle:
data = data[data["SurrogatId"]==0]
for index, rowsource in data.iterrows(): 
#   gibt es schon einen Eintrag mit diesem Business Key?    
    mask =  df_dim["BusinessKey"]==rowsource["BusinessKey"]
    if len(df_dim[mask]==1):
#     Eintrag gibt es schon, iloc[0] macht aus der ersten Zeile eien Series        
      rowdim = df_dim[mask].iloc[0]   
#     Attribut geändert?    
      if (rowdim["Salesorg"] != rowsource["Salesorg"]):
#     Eintrag schon vorhanden und Attribut geändert    
        data.at[index,"ScdVersion"]=rowdim["ScdVersion"]+1
        data.at[index,"SurrogatId"] = surrogatnext
        surrogatnext = surrogatnext + 1
        data.at[index,"Current"]=1
        data.at[index,"Valid To"]= pd.Timestamp(year=2099,month=12,day=31)  
#       zu ändernder alter Eintrag sammeln:
        rowdim["Current"]=0
        rowdim["ValidTo"] = rowsource["ValidFrom"] - pd.Timedelta(days=1)
        rows.append(rowdim) 
      else: 
#       zeile muss aus dem delta gelöscht werden
        data.drop(index, inplace=True)
    else:
#     Business Key nicht vorhanden
      data.at[index,"ScdVersion"]=1 
      data.at[index,"SurrogatId"] = surrogatnext
      surrogatnext = surrogatnext + 1
      data.at[index,"Current"]=1
      data.at[index,"ValidTo"]= pd.Timestamp(year=2099,month=12,day=31)     
    
# geänderte alte Einträge dazu   
data = data.append(rows, ignore_index=True) 


In [ ]:
data

,SurrogatId,BusinessKey,Description,Currency,Salesorg,Country,SCD Version,Current,Valid From,Valid To
0,4,6000,Windy City Bikes,USD,UW00,US,2,1,2014-01-01,2099-12-31
1,5,8000,Motown Bikes,USD,UE00,US,1,1,2014-01-01,2099-12-31
2,2,6000,Windy City Bikes,USD,UE00,US,1,0,1900-01-01,2013-12-31


In [ ]:
data.dtypes

SurrogatId              int64
BusinessKey             int64
Description            object
Currency               object
Salesorg               object
Country                object
ScdVersion              int64
Current                 int64
ValidFrom      datetime64[ns]
ValidTo        datetime64[ns]
dtype: object